In [2]:
import reprodICU
import polars as pl

labs = reprodICU.timeseries_labs
info = reprodICU.patient_information

In [2]:
print(info.select("Global ICU Stay ID").unique().collect().to_series().len())

356849


In [20]:
print(f"patient_info : {info.select("Global ICU Stay ID").unique().collect().to_series().len()}")
print(f"labs : {labs.select("Global ICU Stay ID").unique().collect().to_series().len()}")

patient_info : 356849
labs : 348843


In [23]:
print("Age None", info.filter(pl.col("Admission Age (years)").is_null()).collect().to_series().len())
print("Adults", info.filter(pl.col("Admission Age (years)") >= 18).collect().to_series().len())
print("Minors", info.filter(pl.col("Admission Age (years)") < 18).collect().to_series().len())


Age None 95
Adults 348040
Minors 8714


In [ ]:
labs.select("Global ICU Stay ID", "pH").filter(pl.col("pH").is_not_null()).collect()

print(f"Lab values : {labs.select("pH").collect().to_series().len()}")
print(f"pH values : {labs.filter(pl.col("pH").is_not_null()).select("pH").collect().to_series().len()}")
print(f"patients with pH values : {labs.filter(pl.col("pH").is_not_null()).select("Global ICU Stay ID").unique().collect().to_series().len()}")
print(f"patients with pH values <= 7.2 : {labs.filter(pl.col("pH").struct.field("value") <= 7.2).select("Global ICU Stay ID").unique().collect().to_series().len()}")
print(f"patients with CO2 values <= 45 : {labs.filter(pl.col("Carbon dioxide").struct.field("value") <= 45).select("Global ICU Stay ID").unique().collect().to_series().len()}")
print(f"patients with Bicarb values <= 20 : {labs.filter(pl.col("Bicarbonate").struct.field("value") <= 20).select("Global ICU Stay ID").unique().collect().to_series().len()}")
print(f"patients with (one LAB) pH <= 7.2, CO2 <= 45, Bicarb <= 20 : {labs.filter(
        (pl.col("pH").struct.field("value") <= 7.2) & (pl.col("Carbon dioxide").struct.field("value") <= 45) & (pl.col("Bicarbonate").struct.field("value") <= 20)
    ).select("Global ICU Stay ID").unique().collect().to_series().len()}")
print(f"patients with (any LAB) pH <= 7.2, CO2 <= 45, Bicarb <= 20 : {labs.filter(
        (pl.col("pH").struct.field("value") <= 7.2) & (pl.col("Carbon dioxide").struct.field("value") <= 45).any().over("Global ICU Stay ID") & (pl.col("Bicarbonate").struct.field("value") <= 20).any().over("Global ICU Stay ID")
    ).select("Global ICU Stay ID").unique().collect().to_series().len()}")
print(f"patients with (one LAB) pH <= 7.2, CO2 <= 45, Bicarb <= 20 : <48h {labs.filter(
        (pl.col("Time Relative to Admission (seconds)") <= 48 * 3600) &
        (pl.col("pH").struct.field("value") <= 7.2) & (pl.col("Carbon dioxide").struct.field("value") <= 45) & (pl.col("Bicarbonate").struct.field("value") <= 20)
    ).select("Global ICU Stay ID").unique().collect().to_series().len()}")
print(f"patients with (any LAB) pH <= 7.2, CO2 <= 45, Bicarb <= 20 : <48h {
        labs.filter(
            (pl.col("Time Relative to Admission (seconds)") <= 48 * 3600) &
            (pl.col("pH").struct.field("value") <= 7.2)
        ).filter(
            (pl.col("Carbon dioxide").struct.field("value") <= 45).any().over("Global ICU Stay ID") & 
            (pl.col("Bicarbonate").struct.field("value") <= 20).any().over("Global ICU Stay ID")
        ).select("Global ICU Stay ID").unique().collect().to_series().len()}")
)




Lab values : 8987748
pH values : 1733967
patients with pH values : 205503
patients with pH values <= 7.2 : 98797
patients with CO2 values <= 45 : 165337
patients with Bicarb values <= 20 : 121841
patients with (one LAB) pH <= 7.2, CO2 <= 45, Bicarb <= 20 : 5898
patients with (any LAB) pH <= 7.2, CO2 <= 45, Bicarb <= 20 : 43108
patients with (one LAB) pH <= 7.2, CO2 <= 45, Bicarb <= 20 : <48h 5280
patients with (any LAB) pH <= 7.2, CO2 <= 45, Bicarb <= 20 : <48h 5890


In [ ]:
x = labs.filter(
            (pl.col("Time Relative to Admission (seconds)") <= 48 * 3600) &
            (pl.col("pH").struct.field("value") <= 7.2)
        ).filter(
            (pl.col("Carbon dioxide").struct.field("value") <= 45).any().over("Global ICU Stay ID") & 
            (pl.col("Bicarbonate").struct.field("value") <= 20).any().over("Global ICU Stay ID")
        ).select("Global ICU Stay ID").unique().collect().to_series()
print("Labs : ", x.len())
y = info.filter(pl.col("Global ICU Stay ID").is_in(x)).collect()
print("Info : ", y.select("Global ICU Stay ID").unique().to_series().len())


Labs :  5890
Info :  1308


In [ ]:
print(info.select("Global Hospital Stay ID").unique().collect().to_series().len())
print(info.select("Global ICU Stay ID").unique().collect().to_series().len())
print(labs.select("Global ICU Stay ID").unique().collect().to_series().len())
print(
    info.select("Global ICU Stay ID").unique().join(
        labs.select("Global ICU Stay ID").unique(),
        on="Global ICU Stay ID",
        how="anti"
    ).collect()
)



309383
356849
348843
shape: (203_345, 1)
┌────────────────────┐
│ Global ICU Stay ID │
│ ---                │
│ str                │
╞════════════════════╡
│ eicu-762625        │
│ eicu-1575215       │
│ eicu-2430009       │
│ eicu-2814920       │
│ eicu-3341016       │
│ …                  │
│ eicu-3218038       │
│ eicu-370302        │
│ eicu-1688285       │
│ eicu-3152402       │
│ eicu-1615058       │
└────────────────────┘


In [ ]:
eicu_lab = labs.filter(pl.col("Global ICU Stay ID").str.starts_with("eicu"))
print(eicu_lab.select("Global ICU Stay ID").unique().collect().to_series().len())
print(eicu_lab.filter(pl.col("pH").is_not_null()).select("Global ICU Stay ID").unique().collect().to_series().len())

195339
83752


In [ ]:
eicu = info.filter(pl.col("Global ICU Stay ID").str.starts_with("eicu"))
print(eicu.select("Global ICU Stay ID").unique().collect().to_series().len())



200859


In [ ]:
x = info.select("Global ICU Stay ID").unique().join(
        labs.select("Global ICU Stay ID").unique(),
        on="Global ICU Stay ID",
        how="anti"
    )
print(x.collect().to_series().len())
print(
    x.filter(pl.col("Global ICU Stay ID").str.starts_with("eicu")).collect()
)
# There are a lot of labs in EICU without a definition in patient info ??? 
# but there are +- same number of eicu patients in patient_info and ts_labs !?
# mapping issue ?? 



203345
shape: (200_859, 1)
┌────────────────────┐
│ Global ICU Stay ID │
│ ---                │
│ str                │
╞════════════════════╡
│ eicu-2292199       │
│ eicu-1276845       │
│ eicu-1668734       │
│ eicu-632858        │
│ eicu-3345015       │
│ …                  │
│ eicu-3082703       │
│ eicu-160897        │
│ eicu-2958690       │
│ eicu-1180699       │
│ eicu-3155344       │
└────────────────────┘


In [ ]:
x = info.select("Global ICU Stay ID").unique().collect()
y = labs.select("Global ICU Stay ID").unique().collect()

print("Labs in info : ", x.filter(pl.col("Global ICU Stay ID").is_in(y.to_series())).to_series().len())
print("Info in labs : ", y.filter(pl.col("Global ICU Stay ID").is_in(x.to_series())).to_series().len())

: 